In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import csv
from datetime import datetime
import os

from pathlib import Path
from pprint import pprint

import pandas as pd
from pandas import DataFrame

import json

import graphsense
from graphsense.api import addresses_api, blocks_api, entities_api, general_api, txs_api, bulk_api

## Loading seed and target data

In [4]:
seed_address = '1BswBQiQrUMk3efFdTFZfnsgodB7xjrcGc'
target_address = '33qpZENhgeX3huCzVGDD5NNKr9rK5NcMD9'

## Establish GS Connection

In [7]:
f = open('config.json')
config = json.load(f)
f.close()

In [8]:
configuration = graphsense.Configuration(
    host = config['graphsense']['host'],
    api_key = {'api_key': config['graphsense']['api_key']})

CURRENCY = 'btc'

## Fetch address details

In [11]:
def fetch_address_details(address):
    with graphsense.ApiClient(configuration) as api_client:
        try:
            api_instance = addresses_api.AddressesApi(api_client)

            address_details = api_instance.get_address(CURRENCY, address)
            return address_details

        except graphsense.ApiException as e:
            print("Exception when calling Bulk Api: %s\n" % e)


In [15]:
seed_addr_details = fetch_address_details(seed_address)
seed_addr_details

{'address': '1BswBQiQrUMk3efFdTFZfnsgodB7xjrcGc',
 'balance': {'fiat_values': [{'code': 'eur', 'value': 0.0},
                             {'code': 'usd', 'value': 0.0}],
             'value': 0},
 'currency': 'btc',
 'entity': 435755627,
 'first_tx': {'height': 547167,
              'timestamp': 1540406274,
              'tx_hash': 'e2606176793515d476ac63f63ac0ec8cc75af46ca4c5a466695122ad50462364'},
 'in_degree': 1,
 'last_tx': {'height': 576808,
             'timestamp': 1558296707,
             'tx_hash': 'b09aa1531c41dac74df2bf383955ea767954f6086a0be116efb9d5394d096408'},
 'no_incoming_txs': 1,
 'no_outgoing_txs': 1,
 'out_degree': 1,
 'status': 'clean',
 'total_received': {'fiat_values': [{'code': 'eur', 'value': 0.35},
                                    {'code': 'usd', 'value': 0.4}],
                    'value': 6156},
 'total_spent': {'fiat_values': [{'code': 'eur', 'value': 0.45},
                                 {'code': 'usd', 'value': 0.5}],
                 'value': 6156}

In [17]:
target_addr_details = fetch_address_details(target_address)
target_addr_details

{'address': '33qpZENhgeX3huCzVGDD5NNKr9rK5NcMD9',
 'balance': {'fiat_values': [{'code': 'eur', 'value': 0.0},
                             {'code': 'usd', 'value': 0.0}],
             'value': 0},
 'currency': 'btc',
 'entity': 487558634,
 'first_tx': {'height': 576927,
              'timestamp': 1558349913,
              'tx_hash': 'a032e399a9f7baa9ec36b5f20c0a93dbf2df018255b662db499d8e65c675d3d6'},
 'in_degree': 1,
 'last_tx': {'height': 576929,
             'timestamp': 1558350963,
             'tx_hash': '17d71751a415266368fe68e22de3f8051412fc8c01153ebe487b37f62494f279'},
 'no_incoming_txs': 1,
 'no_outgoing_txs': 1,
 'out_degree': 2,
 'status': 'clean',
 'total_received': {'fiat_values': [{'code': 'eur', 'value': 1278.79},
                                    {'code': 'usd', 'value': 1428.02}],
                    'value': 17898818},
 'total_spent': {'fiat_values': [{'code': 'eur', 'value': 1278.79},
                                 {'code': 'usd', 'value': 1428.02}],
             

## Search neighbors

In [18]:
def get_addr_neighbors(address):
    
    with graphsense.ApiClient(configuration) as api_client:
        try:
            api_instance = bulk_api.BulkApi(api_client)
            print(f"get_addr_neighbors of {address}")
            operation = "list_address_neighbors"
            body = {'address': [address], 'direction': 'out'}

            df_address_neighbors = pd.read_csv(api_instance.bulk_csv(CURRENCY, operation, body=body,
                                                                   num_pages=1, _preload_content=False))
            
            df_address_neighbors = df_address_neighbors \
                .loc[(df_address_neighbors['_error'] != 'not found') &
                     (df_address_neighbors['_info'] != 'no data')].reset_index(drop=True)
            if df_address_neighbors.empty:
                df_address_neighbors.columns = ['address', 'entity', 'out_degree']
                return df_address_neighbors

            
            df_address_neighbors = df_address_neighbors \
                .rename(columns={'address_address': 'address', 
                                 'address_entity': 'entity',
                                 'address_out_degree': 'out_degree'}) \

            return df_address_neighbors[['address', 'entity', 'out_degree']]
            
        except graphsense.ApiException as e:
            print("Exception when calling Bulk Api: %s\n" % e)

In [22]:
def bfs(seed_address,
        target_addresses = [],
        target_entities = [],
        max_depth = 3, max_outdegree = 10, verbose = False):
    
    # collect matching paths
    matching_paths = []
    
    # record visited addresses and entities
    visited_entities = []
    
    # maintain a queue of paths
    queue = []
    
    # push the first path into the queue
    queue.append([seed_address])
    
    # count number of requests
    no_requests = 0
    
    while(queue):

        # get first path from the queue
        path = queue.pop(0)
        print(f"No requests: {no_requests}, " +
              f"Queue size: {len(queue)}, " +
              f"path length: {len(path)}, " +
              f"seed address {seed_address}", end='\r')
        
        # get the last node from the path
        addr = path[-1]

        # retrieve address neighbors
        df_neighbors = get_addr_neighbors(addr)
        no_requests = no_requests + 1

        # continue with neighbors out_degree < max_outdegree
        for index, neighbor in df_neighbors.iterrows():

            new_path = list(path)
            new_path.append(neighbor['address'])

            # found path (address match)
            if(neighbor['address'] in target_addresses):
                print(new_path, end=' ')
                print("MATCH | address")
                matching_paths.append(new_path)
                continue

            # found path (entity match)
            if(neighbor['entity'] in target_entities):
                print(new_path, end=' ')
                print("MATCH | entity")
                matching_paths.append(new_path)
                continue
                                
            # stop if max depth is reached
            if len(new_path) == max_depth:
                if verbose:
                    print(new_path, end=' ') 
                    print("STOP | max depth")
                continue
            
            # stop if entity has already been visited
            if(neighbor['entity'] in visited_entities):
                if verbose:
                    print(new_path, end=' ') 
                    print("STOP | same entity")
                continue

            # stop if address out_degree exceeds threshold
            if(neighbor['out_degree'] > max_outdegree):
                if verbose:
                    print(new_path, end=' ') 
                    print("STOP | max outdegree")
                continue

            queue.append(new_path)
            visited_entities.append(neighbor['entity'])
                
        if len(queue) == 0:
            return matching_paths

# Applying all the stuff

### Test example (address match)

In [25]:
paths = bfs(seed_address,
            [target_address],
            [],
            7, 20, False)

get_addr_neighbors of 1BswBQiQrUMk3efFdTFZfnsgodB7xjrcGcess 1BswBQiQrUMk3efFdTFZfnsgodB7xjrcGc
get_addr_neighbors of bc1qhlsngxttkqa2pl6ahjjn8h4qhgut9ajetrddq7BQiQrUMk3efFdTFZfnsgodB7xjrcGc
get_addr_neighbors of 1CPXbR6axkHXxNhWrtouRcu13cN77ccZDZess 1BswBQiQrUMk3efFdTFZfnsgodB7xjrcGc
get_addr_neighbors of bc1qj2a6a7xqnf5pkc3spjnhm224rem4mxvphu8eeaBQiQrUMk3efFdTFZfnsgodB7xjrcGc
get_addr_neighbors of 3Bjo93bKcNk5nL1vNPr4hp7cWfGd77Hhv7ess 1BswBQiQrUMk3efFdTFZfnsgodB7xjrcGc
get_addr_neighbors of 34SBebUu7FjqHAGP9LiPjjtZbs5cFzisHGess 1BswBQiQrUMk3efFdTFZfnsgodB7xjrcGc
get_addr_neighbors of bc1qru49trsw24yqkrc4yh7kgu8u5crta06zlhdvrrBQiQrUMk3efFdTFZfnsgodB7xjrcGc
get_addr_neighbors of bc1qkwuutl6lk9q9j38qhnrzk35rqqnjhmazgugy0vBQiQrUMk3efFdTFZfnsgodB7xjrcGc
get_addr_neighbors of 39HSKGYcb3VguLbd7f8TBd6Nd3oAKeKcjJess 1BswBQiQrUMk3efFdTFZfnsgodB7xjrcGc
get_addr_neighbors of 17yZaZjE4jQzygSbcgPEKMZxqagWNk7rBKess 1BswBQiQrUMk3efFdTFZfnsgodB7xjrcGc
get_addr_neighbors of bc1qa0lejnyttw5zahr9ytlagd0y

Copy and paste the path into the dashboard:

In [26]:
' '.join(paths[0])

'1BswBQiQrUMk3efFdTFZfnsgodB7xjrcGc bc1qhlsngxttkqa2pl6ahjjn8h4qhgut9ajetrddq7 bc1qkwuutl6lk9q9j38qhnrzk35rqqnjhmazgugy0v bc1q78yrdssz4tj0tr7xx5sku5vxjr7wj3gud43ahv 33qpZENhgeX3huCzVGDD5NNKr9rK5NcMD9'